In [17]:
# Packages

import pandas as pd
import numpy as np
import scipy
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot

from scipy.special import beta
from scipy import optimize

import warnings
warnings.filterwarnings("ignore")

import os
WF=os.getcwd()+'/'

## Read Data

In [20]:
raw=pd.read_excel(open(WF+'input.xlsx', 'rb'),sheet_name='Query') 
raw=raw.groupby(['STARTMONTH','MONTHS_SINCE_START','INDICATION_CODED','AFTER_LAUNCH_FLAG'],as_index=False)['PTS_ON_TX_IN_MONTH_60'].sum()
raw=raw.rename(columns={'STARTMONTH':'Cohort','MONTHS_SINCE_START':'month_passed','PTS_ON_TX_IN_MONTH_60':'patients','INDICATION_CODED':'indication'})
raw[:1]

,Cohort,month_passed,indication,AFTER_LAUNCH_FLAG,patients
0,201704,0,AD Adolescents,0,4


## Model comparison

In [14]:
output=[]
for ind in raw['indication'].unique():
    print(ind)
    for cohort in raw[(raw['indication']==ind)]['Cohort'].unique():
        print(cohort)
        df=raw[(raw['Cohort']==cohort) & (raw['indication']==ind)]
        df=df.sort_values('month_passed',ascending=True).reset_index()
        
        for f in pred_range_model_eval:
            test=df[(-f):][['month_passed','patients']]
            train=df[:(-f)][['month_passed','patients']]
            #train['net']=train['patients']-train['patients'].shift(-1)

            def expo(t,a):
                s=np.power(a,t-1)
                return(train['patients'].max()*s)
            try:
                params_exp, params_exp_cov=optimize.curve_fit(expo, train['month_passed'],train['patients'] )
                EXPO=pd.DataFrame({'month_passed':df['month_passed'],'pred_expo':expo(df['month_passed'], *params_exp)})
            except:
                EXPO=pd.DataFrame({'month_passed':df['month_passed'],'pred_expo':np.nan})
                
            def bg(t,a,b):
                s=beta(a,b+t-1)/beta(a,b)
                return(train['patients'].max()*s)
            try:
                params_bg, params_bg_cov = optimize.curve_fit(bg, train['month_passed'],train['patients'] )
                BG=pd.DataFrame({'month_passed':df['month_passed'],'pred_bg':bg(df['month_passed'], *params_bg)})
            except:
                BG=pd.DataFrame({'month_passed':df['month_passed'],'pred_bg':np.nan})

            def bdw(t,a,b,c):
                s=beta(a,b+np.power(t-1,c))/beta(a,b)
                return(train['patients'].max()*s)
            try:
                params_bdw, params_bdw_cov = optimize.curve_fit(bdw, train['month_passed'],train['patients'] )
                BDW=pd.DataFrame({'month_passed':df['month_passed'],'pred_bdw':bdw(df['month_passed'], *params_bdw)})
            except:
                BDW=pd.DataFrame({'month_passed':df['month_passed'],'pred_bdw':np.nan})
                
            def lcdw(t,w,a1,b1,c1,a2,b2,c2):
                s=w*beta(a1,b1+np.power(t-1,c1))/beta(a1,b1)+(1-w)*beta(a2,b2+np.power(t-1,c2))/beta(a2,b2)
                return(train['patients'].max()*s)
            try:
                params_lcdw, params_lcdw_cov = optimize.curve_fit(lcdw, train['month_passed'],train['patients'],maxfev = 5000 )
                LCDW=pd.DataFrame({'month_passed':df['month_passed'],'pred_lcdw':lcdw(df['month_passed'], *params_lcdw)})
            except:
                LCDW=pd.DataFrame({'month_passed':df['month_passed'],'pred_lcdw':np.nan})

            Pred=EXPO.merge(BG,how='outer',on='month_passed').merge(BDW,how='outer',on='month_passed').merge(LCDW,how='outer',on='month_passed').merge(df[['month_passed','patients']],how='outer',on='month_passed')
            Pred['Out_Of_Sample']=Pred['month_passed']>Pred['month_passed'].max()-f

            for x in [x for x in list(Pred.columns) if x.startswith('pred_')]:
                Pred[x.replace('pred_','pe_')]=Pred[x]/Pred['patients']-1
                Pred[x.replace('pred_','ape_')]=Pred[x.replace('pred_','pe_')].map(lambda x: np.abs(x))
                output.append([ind,cohort,f,len(df['month_passed'])-f,x.replace('_pred',''),
                                            Pred[Pred['Out_Of_Sample']==True][x.replace('pred_','pe_')].mean(),
                                            Pred[Pred['Out_Of_Sample']==True][x.replace('pred_','ape_')].mean()])       

AD Adolescents
2017-04-01
2017-05-01
2017-06-01
2017-07-01
2017-08-01
2017-09-01
2017-10-01
2017-11-01
2017-12-01
2018-01-01
2018-02-01
2018-03-01
2018-04-01
2018-05-01
2018-06-01
2018-07-01
2018-08-01
2018-09-01
2018-10-01
2018-11-01
2018-12-01
2019-01-01
2019-02-01
2019-03-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01
2019-10-01
2019-11-01
2019-12-01
2020-01-01
2020-02-01
2020-03-01
2020-04-01
2020-05-01
2020-06-01
2020-07-01
2020-08-01
2020-09-01
2020-10-01
2020-11-01
2020-12-01
AD Adults
2017-04-01
2017-05-01
2017-06-01
2017-07-01
2017-08-01
2017-09-01
2017-10-01
2017-11-01
2017-12-01
2018-01-01
2018-02-01
2018-03-01
2018-04-01
2018-05-01
2018-06-01
2018-07-01
2018-08-01
2018-09-01
2018-10-01
2018-11-01
2018-12-01
2019-01-01
2019-02-01
2019-03-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01
2019-10-01
2019-11-01
2019-12-01
2020-01-01
2020-02-01
2020-03-01
2020-04-01
2020-05-01
2020-06-01
2020-07-01
2020-08-01
2020-09-01
2020-10-01
2020-11

In [15]:
output=pd.DataFrame(output,columns=['Indication','Cohort','Out_Of_Sample','Sample','Method','PE','APE'])
output['WAPE']=output['Out_Of_Sample']*output['APE']
output[:3]

,Indication,Cohort,Out_Of_Sample,Sample,Method,PE,APE,WAPE
0,AD Adolescents,2017-04-01,1,44,pred_expo,0.070506,0.070506,0.070506
1,AD Adolescents,2017-04-01,1,44,pred_bg,0.071701,0.071701,0.071701
2,AD Adolescents,2017-04-01,1,44,pred_bdw,NaN,NaN,NaN


In [16]:
model_evaluation=output.copy()
model_evaluation=model_evaluation[(model_evaluation['Sample']>5) & (model_evaluation['APE']!=np.inf)]
me_cohort=model_evaluation.groupby(['Indication','Cohort','Method'],as_index=False)['APE','PE'].mean()
me_outofsample=model_evaluation.groupby(['Indication','Out_Of_Sample','Method'],as_index=False)['APE','PE'].mean()
me_smy=model_evaluation.groupby(['Indication','Method'],as_index=False)['APE','PE'].mean()

In [17]:
import os
if not os.path.exists(WF+'Output/'+latest.strftime('%Y%m%d')):
    os.makedirs(WF+'Output/'+latest.strftime('%Y%m%d'))

In [18]:
writer2= pd.ExcelWriter(WF+'Output/'+latest.strftime('%Y%m%d')+'/Output_Model_Evaluation_'+latest.strftime('%Y%m%d')+'.xlsx')
for ind in model_evaluation['Indication'].unique():
    writer = pd.ExcelWriter(WF+'Output/'+latest.strftime('%Y%m%d')+'/Output_Model_Evaluation_'+latest.strftime('%Y%m%d')+'_'+ind+'.xlsx')
    model_evaluation[model_evaluation['Indication']==ind].to_excel(writer, sheet_name='raw',index=False)
    me_cohort[me_cohort['Indication']==ind].to_excel(writer, sheet_name='cohort',index=False)
    me_outofsample[me_outofsample['Indication']==ind].to_excel(writer, sheet_name='OutOfSample',index=False)
    me_smy[me_smy['Indication']==ind].to_excel(writer, sheet_name='Summary',index=False)
    writer.save()
    
    model_evaluation[model_evaluation['Indication']==ind].to_excel(writer2, sheet_name='raw_'+ind,index=False)
    me_cohort[me_cohort['Indication']==ind].to_excel(writer2, sheet_name='cohort_'+ind,index=False)
    me_outofsample[me_outofsample['Indication']==ind].to_excel(writer2, sheet_name='OutOfSample_'+ind,index=False)
    me_smy[me_smy['Indication']==ind].to_excel(writer2, sheet_name='Summary_'+ind,index=False)
writer2.save()

## Prediction

In [1]:
output=[]
for ind in raw['indication'].unique():
    print(ind)
    for cohort in raw[(raw['indication']==ind)]['Cohort'].unique(): #[x for x in raw['Cohort'].unique() if x=='2017-12-01']: # 
        print(cohort)
        df=raw[(raw['Cohort']==cohort) & (raw['indication']==ind)]
        df=df.sort_values('month_passed',ascending=True).reset_index()

        def bdw(t,a,b,c):
            s=beta(a,b+np.power(t-1,c))/beta(a,b)
            return(np.log(df['patients'].max()*s))
        try:
            params_bdw, params_bdw_cov = optimize.curve_fit(bdw, df['month_passed'],df['patients'].map(lambda x:np.log(x)) )
            BDW=pd.DataFrame({'month_passed':df['month_passed'],'pred_bdw':bdw(df['month_passed'], *params_bdw)})
        except:
            BDW=pd.DataFrame({'month_passed':df['month_passed'],'pred_bdw':np.nan})

        def bg(t,a,b):
            s=beta(a,b+t-1)/beta(a,b)
            return(np.log(df['patients'].max()*s))
        try:
            params_bg, params_bg_cov = optimize.curve_fit(bg, df['month_passed'],df['patients'].map(lambda x:np.log(x)) )
            BG=pd.DataFrame({'month_passed':df['month_passed'],'pred_bg':bg(df['month_passed'], *params_bg)})
        except:
            BG=pd.DataFrame({'month_passed':df['month_passed'],'pred_bg':np.nan})

        def lcdw(t,w,a1,b1,c1,a2,b2,c2):
            s=w*beta(a1,b1+np.power(t-1,c1))/beta(a1,b1)+(1-w)*beta(a2,b2+np.power(t-1,c2))/beta(a2,b2)
            return(np.log(df['patients'].max()*s))
        try:
            params_lcdw, params_lcdw_cov = optimize.curve_fit(lcdw, df['month_passed'],df['patients'].map(lambda x:np.log(x)),maxfev = 5000 )
            LCDW=pd.DataFrame({'month_passed':df['month_passed'],'pred_lcdw':lcdw(df['month_passed'], *params_lcdw)})
        except:
            LCDW=pd.DataFrame({'month_passed':df['month_passed'],'pred_lcdw':np.nan})

        def expo(t,a):
            s=np.power(a,t-1)
            return(np.log(df['patients'].max()*s))
        try:
            params_exp, params_exp_cov=optimize.curve_fit(expo, df['month_passed'],df['patients'].map(lambda x:np.log(x)) )
            EXPO=pd.DataFrame({'month_passed':df['month_passed'],'pred_expo':expo(df['month_passed'], *params_exp)})
        except:
            EXPO=pd.DataFrame({'month_passed':df['month_passed'],'pred_expo':np.nan})

        Pred=pred_range.copy()
        Pred['month_passed']=Pred['month_passed']+df['month_passed'].max()
        Pred['patients_BG']=Pred['month_passed'].map(lambda x: np.exp(bg(x,*params_bg)))
        Pred['patients_Bdw']=Pred['month_passed'].map(lambda x: np.exp(bdw(x,*params_bdw)))
        Pred['patients_LCDW']=Pred['month_passed'].map(lambda x: np.exp(lcdw(x,*params_lcdw)))
        Pred['patients_Expo']=Pred['month_passed'].map(lambda x: np.exp(expo(x,*params_exp)))
        Pred['Out_Of_Sample']=True
        Pred['Cohort']=cohort
        Pred['Indication']=ind

        History=df[['Cohort','month_passed','patients']]
        History['patients_BG']=History['patients']
        History['patients_Bdw']=History['patients']
        History['patients_LCDW']=History['patients']
        History['patients_Expo']=History['patients']
        History['Out_Of_Sample']=False
        History=History.drop('patients',1)
        History['Indication']=ind

        if type(output)==list:
            output=pd.concat([History,Pred])
        else:
            output=pd.concat([output,History,Pred])

In [20]:
output['tmp']=output['Cohort'].map(lambda x: [int(y) for y in x.split('-')])
output['cohort_month']=output['tmp'].map(lambda x: datetime.datetime(x[0],x[1],x[2]))
output['month']=output.apply(lambda x: x['cohort_month']+relativedelta(months=x['month_passed']),1)
output[:3]

,Cohort,month_passed,patients_BG,patients_Bdw,patients_LCDW,patients_Expo,Out_Of_Sample,Indication,tmp,cohort_month,month
0,2017-04-01,1,4.0,4.0,4.0,4.0,False,AD Adolescents,"[2017, 4, 1]",2017-04-01,2017-05-01
1,2017-04-01,2,4.0,4.0,4.0,4.0,False,AD Adolescents,"[2017, 4, 1]",2017-04-01,2017-06-01
2,2017-04-01,3,4.0,4.0,4.0,4.0,False,AD Adolescents,"[2017, 4, 1]",2017-04-01,2017-07-01


In [21]:
pc=output[output['Cohort']<=last.strftime('%Y-%m-%d')]
pc=pc.drop(['tmp','cohort_month'],1)
pc[:3]

,Cohort,month_passed,patients_BG,patients_Bdw,patients_LCDW,patients_Expo,Out_Of_Sample,Indication,month
0,2017-04-01,1,4.0,4.0,4.0,4.0,False,AD Adolescents,2017-05-01
1,2017-04-01,2,4.0,4.0,4.0,4.0,False,AD Adolescents,2017-06-01
2,2017-04-01,3,4.0,4.0,4.0,4.0,False,AD Adolescents,2017-07-01


In [22]:
AD=pc[pc['Indication'].isin(['AD Adolescents','AD Adults','AD Pediatrics'])]
AD=AD.groupby(['Cohort','month_passed','Out_Of_Sample','month'],as_index=False)['patients_BG','patients_Bdw','patients_LCDW','patients_Expo'].sum()
AD['Indication']='AD Overall V2'
pc=pd.concat([pc,AD])
AD[:2]

,Cohort,month_passed,Out_Of_Sample,month,patients_BG,patients_Bdw,patients_LCDW,patients_Expo,Indication
0,2017-04-01,1,False,2017-05-01,295.0,295.0,295.0,295.0,AD Overall V2
1,2017-04-01,2,False,2017-06-01,293.0,293.0,293.0,293.0,AD Overall V2


## Persistency Calculator

In [ ]:
pc.to_csv(path+'/pc.csv',index=False)

In [23]:
# Weighted persistency
output2=[]
for ind in pc['Indication'].unique():
    for m in persistence_range:
        P=['patients_BG','patients_Bdw','patients_LCDW','patients_Expo']
        ag=pc[(pc['month_passed']<m) & (pc['Indication']==ind)].groupby('month_passed',as_index=False)[P].sum()
        for p in P:
            ag[p+'_net']=ag[p]-ag[p].shift(-1)
            ag.loc[ag[p+'_net'].isnull(),p+'_net']=ag[p]
            ag[p+'_w']=ag[p+'_net']*ag['month_passed']
            output2.append([ind,m,p,ag[p+'_w'].sum()/ag[ag['month_passed']==1]['patients_BG'].max()])
output2=pd.DataFrame(output2,columns=['Indication','Look_Forward','Methodology','Persistence'])
output2['Average']='Weighted'
output2[:3]

,Indication,Look_Forward,Methodology,Persistence,Average
0,AD Adolescents,12,patients_BG,9.347406,Weighted
1,AD Adolescents,12,patients_Bdw,9.346201,Weighted
2,AD Adolescents,12,patients_LCDW,9.292878,Weighted


In [24]:
writer2= pd.ExcelWriter(WF+'Output/'+latest.strftime('%Y%m%d')+'/Output_Persistency_'+latest.strftime('%Y%m%d')+'.xlsx')
for ind in pc['Indication'].unique():
    writer = pd.ExcelWriter(WF+'Output/'+latest.strftime('%Y%m%d')+'/Output_Persistency_'+latest.strftime('%Y%m%d')+'_'+ind+'.xlsx')
    pc[pc['Indication']==ind].to_excel(writer, sheet_name='predication',index=False)
    output2[output2['Indication']==ind].to_excel(writer, sheet_name='persistency',index=False)
    writer.save()
    
    pc[pc['Indication']==ind].to_excel(writer2, sheet_name='predication_'+ind,index=False)
    output2[output2['Indication']==ind].to_excel(writer2, sheet_name='persistency_'+ind,index=False)
writer2.save()

## Copy input file for record

In [25]:
from shutil import copyfile
copyfile(WF+'input.xlsx', WF+'/Input/input_'+latest.strftime('%Y%m%d')+'.xlsx')

'/Users/can.tan/Documents/Project/Dupixent/Persistence Calculator/\\Input\\input_20210101.xlsx'

## output one sheeter

In [26]:
persistence=[]
model_evaluation=[]
for ind in pc['Indication'].unique():
    df=pd.read_excel((WF+'Output\\'+latest.strftime('%Y%m%d')+'\\Output_Persistency_'+latest.strftime('%Y%m%d')+'.xlsx'),sheet_name='persistency_'+ind)
    if type(persistence)==list:
        persistence=df
    else:
        persistence=pd.concat([persistence,df])
        
    try:
        df2=pd.read_excel((WF+'Output\\'+latest.strftime('%Y%m%d')+'\\Output_Model_Evaluation_'+latest.strftime('%Y%m%d')+'.xlsx'), sheet_name='Summary_'+ind)
        if type(model_evaluation)==list:
            model_evaluation=df2
        else:
            model_evaluation=pd.concat([model_evaluation,df2])
    except:
        print(ind+' is not available for model evaluation.')

AD Adolescents is not available for model evaluation.
AD Adults is not available for model evaluation.
AD Pediatrics is not available for model evaluation.
Asthma is not available for model evaluation.
NP is not available for model evaluation.
AD Overall is not available for model evaluation.
AD Overall V2 is not available for model evaluation.


In [27]:
writer3= pd.ExcelWriter(WF+'Output\\'+latest.strftime('%Y%m%d')+'\\summary_'+latest.strftime('%Y%m%d')+'.xlsx')
persistence.to_excel(writer3,sheet_name='persistence',index=False)
model_evaluation.to_excel(writer3,sheet_name='model_evaluation',index=False)
writer3.save()

AttributeError: 'list' object has no attribute 'to_excel'